# spacetrackのテスト その1

spacetrack パッケージを用いると、簡単に space-track.org からデータを取得できる。
see https://pythonhosted.org/spacetrack/usage.html

TLE, 3LE, CSV, XML, KVN, JSON (detault), HTML 形式を利用できる。

CSV, XML, KVN, JSON, HTML形式では、TLE, 3LEから以下の情報が追加されている。
- OBJECT_ID : 国際衛星識別符号 (INTLDES のフォーマットを整えたもの)
- OBJECT_NUMBER : NORAD_CAT_IDと基本的に同じ (INTLDESが振られていないものは、OBJECT_NUMBERも振られていない)
- SEMIMAJOR_AXIS (a) = (mu/(n\*2\*pi/(24\*3600))^2)^(1/3)  # mu = 398600.4418 地球の重力
- PERIOD = 1440/n
- APOGEE = (a * (1 + e)) - 6378.135 # 地球は完全な球体だと仮定している
- PERIGEE = (a * (1 - e)) - 6378.135
- DECAYED : Predicted and historical decay information. 

tle系APIで取得できるデータの項目: https://www.space-track.org/basicspacedata/modeldef/class/tle/format/html

https://www.space-track.org/documentation#faq によると、現在では tle, tle_latest, tle_publish API は非推奨となっており、今後は gp, gp_history の利用が推奨される。

In [1]:
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
import pandas as pd
import json
import time

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", 80)

In [3]:
# userid and password are set in here
import spacetrackaccount
# print("Login with " + spacetrackaccount.userid)

In [4]:
st = SpaceTrackClient(spacetrackaccount.userid, spacetrackaccount.password)

In [5]:
def mycallback(until):
    duration = int(round(until - time.monotonic()))
    print('Sleeping for {:d} seconds.'.format(duration))

st.callback = mycallback

In [6]:
#取得したデータを1行ごとに処理する例
#data = st.tle_latest(iter_lines=True, ordinal=1, epoch='>now-30',
#                     mean_motion=op.inclusive_range(0.99, 1.01),
#                     eccentricity=op.less_than(0.01), format='tle')
#with open('tle_latest.tle', 'w') as fp:
#    fp.write(data)
#        fp.write(line + '\n')

##  全衛星の最新データをいろいろな形式でダウンロードする
TLE, 3LE, CSV, XML, KVN, JSON (detault), HTML

In [7]:
# TLE
data_tle = st.tle_latest(ordinal=1, epoch='>now-30', orderby='norad_cat_id', format='tle')
with open('download/tle_latest_30day.tle', 'w') as fp:
    fp.write(data_tle)

In [8]:
# 3LE
data_3le = st.tle_latest(ordinal=1, epoch='>now-30', orderby='norad_cat_id', format='3le')
with open('download/tle_latest_30day.3le', 'w') as fp:
    fp.write(data_3le)

In [9]:
# CSV
data_csv = st.tle_latest(ordinal=1, epoch='>now-30', orderby='norad_cat_id', format='csv')
with open('download/tle_latest_30day.csv', 'w') as fp:
    fp.write(data_csv)

In [10]:
# XML
data_xml = st.tle_latest(ordinal=1, epoch='>now-30', orderby='norad_cat_id', format='xml')
with open('download/tle_latest_30day.xml', 'w') as fp:
    fp.write(data_xml)

In [11]:
# KVN
data_kvn = st.tle_latest(ordinal=1, epoch='>now-30', orderby='norad_cat_id', format='kvn')
with open('download/tle_latest_30day.kvn', 'w') as fp:
    fp.write(data_kvn)

In [12]:
# JSON (default)
data_json = st.tle_latest(ordinal=1, epoch='>now-30', orderby='norad_cat_id', format='json')
with open('download/tle_latest_30day.json', 'w') as fp:
    fp.write(data_json)

In [13]:
# HTML
data_html = st.tle_latest(ordinal=1, epoch='>now-30', orderby='norad_cat_id', format='html')
with open('download/tle_latest_30day.html', 'w') as fp:
    fp.write(data_html)

### 各形式のサイズを比較

In [14]:
# サイズを比較
print('TLE:\t{}\n3LE:\t{}\nCSV:\t{}\nXML:\t{}\nKVN:\t{}\nJSON:\t{}\nHTML:\t{}'
      .format(len(data_tle), len(data_3le), len(data_csv), len(data_xml), len(data_kvn), len(data_json), len(data_html)))

TLE:	2818760
3LE:	3134798
CSV:	9634754
XML:	25423492
KVN:	43127059
JSON:	18782333
HTML:	13843174


### データの中身を確認

In [15]:
# TLEの最初の6行を表示
for line in data_tle.splitlines()[0:6]:
    print(line)

1     5U 58002B   20312.60774602 -.00000017 +00000-0 -42903-4 0  9992
2     5 034.2558 147.2611 1847212 276.5774 062.8942 10.84868483220802
1    11U 59001A   20312.77116121 +.00000226 +00000-0 +11540-3 0  9995
2    11 032.8703 237.3183 1466385 340.5303 014.4488 11.85686439291995
1    12U 59001B   20312.79685902 +.00000303 +00000-0 +18183-3 0  9992
2    12 032.8968 209.4545 1666368 109.5610 269.1976 11.44349894296406


In [16]:
# 3LEの最初の9行を表示
for line in data_3le.splitlines()[0:9]:
    print(line)

0 VANGUARD 1
1     5U 58002B   20312.60774602 -.00000017 +00000-0 -42903-4 0  9992
2     5 034.2558 147.2611 1847212 276.5774 062.8942 10.84868483220802
0 VANGUARD 2
1    11U 59001A   20312.77116121 +.00000226 +00000-0 +11540-3 0  9995
2    11 032.8703 237.3183 1466385 340.5303 014.4488 11.85686439291995
0 VANGUARD R/B
1    12U 59001B   20312.79685902 +.00000303 +00000-0 +18183-3 0  9992
2    12 032.8968 209.4545 1666368 109.5610 269.1976 11.44349894296406


In [17]:
# CSVの最初の4行を表示
for line in data_csv.splitlines()[0:4]:
    print(line)

ORDINAL,COMMENT,ORIGINATOR,NORAD_CAT_ID,OBJECT_NAME,OBJECT_TYPE,CLASSIFICATION_TYPE,INTLDES,EPOCH,EPOCH_MICROSECONDS,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,FILE,TLE_LINE0,TLE_LINE1,TLE_LINE2,OBJECT_ID,OBJECT_NUMBER,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE,DECAYED
"1","GENERATED VIA SPACETRACK.ORG API","18 SPCS","5","VANGUARD 1","PAYLOAD","U","58002B","2020-11-07 14:35:09","256128","10.84868483","0.1847212","34.2558","147.2611","276.5774","62.8942","0","999","22080","-0.000042903","-0.00000017","0","2875047","0 VANGUARD 1","1     5U 58002B   20312.60774602 -.00000017 +00000-0 -42903-4 0  9992","2     5 034.2558 147.2611 1847212 276.5774 062.8942 10.84868483220802","1958-002B","5","8619.533","132.734","3833.609","649.188","0"
"1","GENERATED VIA SPACETRACK.ORG API","18 SPCS","11","VANGUARD 2","PAYLOAD","U","59001A","2020-11-07 18:30:28","328544","11.85686439","0.146638

In [18]:
# XMLを整形して最初の44行を表示
import xml.etree.ElementTree as et
import xml.dom.minidom as md
for line in  md.parseString(et.tostring(et.fromstring(data_xml), 'utf-8')).toprettyxml(indent="    ").splitlines()[0:44]:
    print(line)

<?xml version="1.0" ?>
<xml>
    <item>
        <ORDINAL>1</ORDINAL>
        <COMMENT>GENERATED VIA SPACETRACK.ORG API</COMMENT>
        <ORIGINATOR>18 SPCS</ORIGINATOR>
        <NORAD_CAT_ID>5</NORAD_CAT_ID>
        <OBJECT_NAME>VANGUARD 1</OBJECT_NAME>
        <OBJECT_TYPE>PAYLOAD</OBJECT_TYPE>
        <CLASSIFICATION_TYPE>U</CLASSIFICATION_TYPE>
        <INTLDES>58002B</INTLDES>
        <EPOCH>2020-11-07 14:35:09</EPOCH>
        <EPOCH_MICROSECONDS>256128</EPOCH_MICROSECONDS>
        <MEAN_MOTION>10.84868483</MEAN_MOTION>
        <ECCENTRICITY>0.1847212</ECCENTRICITY>
        <INCLINATION>34.2558</INCLINATION>
        <RA_OF_ASC_NODE>147.2611</RA_OF_ASC_NODE>
        <ARG_OF_PERICENTER>276.5774</ARG_OF_PERICENTER>
        <MEAN_ANOMALY>62.8942</MEAN_ANOMALY>
        <EPHEMERIS_TYPE>0</EPHEMERIS_TYPE>
        <ELEMENT_SET_NO>999</ELEMENT_SET_NO>
        <REV_AT_EPOCH>22080</REV_AT_EPOCH>
        <BSTAR>-0.000042903</BSTAR>
        <MEAN_MOTION_DOT>-0.00000017</MEAN_MOTION_DOT>
      

In [19]:
# KVNの最初の40行を表示
for line in data_kvn.splitlines()[0:40]:
    print(line)

ORDINAL                            =1                        
COMMENT                            =GENERATED VIA SPACETRACK.ORG API
ORIGINATOR                         =18 SPCS                  
NORAD_CAT_ID                       =5                        
OBJECT_NAME                        =VANGUARD 1               
OBJECT_TYPE                        =PAYLOAD                  
CLASSIFICATION_TYPE                =U                        
INTLDES                            =58002B                   
EPOCH                              =2020-11-07 14:35:09      
EPOCH_MICROSECONDS                 =256128                   
MEAN_MOTION                        =10.84868483              
ECCENTRICITY                       =0.1847212                
INCLINATION                        =34.2558                  
RA_OF_ASC_NODE                     =147.2611                 
ARG_OF_PERICENTER                  =276.5774                 
MEAN_ANOMALY                       =62.8942                  
E

In [20]:
# JSONを整形して最初の40行を表示
for line in json.dumps(json.loads(data_json), indent=4).splitlines()[0:40]:
    print(line)

[
    {
        "ORDINAL": "1",
        "COMMENT": "GENERATED VIA SPACETRACK.ORG API",
        "ORIGINATOR": "18 SPCS",
        "NORAD_CAT_ID": "5",
        "OBJECT_NAME": "VANGUARD 1",
        "OBJECT_TYPE": "PAYLOAD",
        "CLASSIFICATION_TYPE": "U",
        "INTLDES": "58002B",
        "EPOCH": "2020-11-07 14:35:09",
        "EPOCH_MICROSECONDS": "256128",
        "MEAN_MOTION": "10.84868483",
        "ECCENTRICITY": "0.1847212",
        "INCLINATION": "34.2558",
        "RA_OF_ASC_NODE": "147.2611",
        "ARG_OF_PERICENTER": "276.5774",
        "MEAN_ANOMALY": "62.8942",
        "EPHEMERIS_TYPE": "0",
        "ELEMENT_SET_NO": "999",
        "REV_AT_EPOCH": "22080",
        "BSTAR": "-0.000042903",
        "MEAN_MOTION_DOT": "-0.00000017",
        "MEAN_MOTION_DDOT": "0",
        "FILE": "2875047",
        "TLE_LINE0": "0 VANGUARD 1",
        "TLE_LINE1": "1     5U 58002B   20312.60774602 -.00000017 +00000-0 -42903-4 0  9992",
        "TLE_LINE2": "2     5 034.2558 147.2611 1

In [21]:
# HTMLを整形して表示
from IPython.display import display, HTML
display(HTML(data_html[0:data_html.find('</tr>', 2500)+5]))

ORDINAL,COMMENT,ORIGINATOR,NORAD_CAT_ID,OBJECT_NAME,OBJECT_TYPE,CLASSIFICATION_TYPE,INTLDES,EPOCH,EPOCH_MICROSECONDS,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,FILE,TLE_LINE0,TLE_LINE1,TLE_LINE2,OBJECT_ID,OBJECT_NUMBER,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE,DECAYED
1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,5,VANGUARD 1,PAYLOAD,U,58002B,2020-11-07 14:35:09,256128,10.84868483,0.1847212,34.2558,147.2611,276.5774,62.8942,0,999,22080,-0.000042903,-0.00000017,0,2875047,0 VANGUARD 1,1 5U 58002B 20312.60774602 -.00000017 +00000-0 -42903-4 0 9992,2 5 034.2558 147.2611 1847212 276.5774 062.8942 10.84868483220802,1958-002B,5,8619.533,132.734,3833.609,649.188,0
1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,11,VANGUARD 2,PAYLOAD,U,59001A,2020-11-07 18:30:28,328544,11.85686439,0.1466385,32.8703,237.3183,340.5303,14.4488,0,999,29199,0.0001154,0.00000226,0,2875047,0 VANGUARD 2,1 11U 59001A 20312.77116121 +.00000226 +00000-0 +11540-3 0 9995,2 11 032.8703 237.3183 1466385 340.5303 014.4488 11.85686439291995,1959-001A,11,8123.726,121.448,2936.842,554.340,0
1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,12,VANGUARD R/B,ROCKET BODY,U,59001B,2020-11-07 19:07:28,619328,11.44349894,0.1666368,32.8968,209.4545,109.561,269.1976,0,999,29640,0.00018183,0.00000303,0,2875047,0 VANGUARD R/B,1 12U 59001B 20312.79685902 +.00000303 +00000-0 +18183-3 0 9992,2 12 032.8968 209.4545 1666368 109.5610 269.1976 11.44349894296406,1959-001B,12,8318.198,125.835,3326.181,553.945,0


### DataFrameに変換する
データの型の自動変換や、欠損値の扱いに注意。

pandas.read_json や pandas.read_csv を用いる際には https://www.space-track.org/basicspacedata/modeldef/class/tle/format/html を参照の上、dtype を指定する方がよい。(以下の例ではdtypeは指定していない)

INTLDES, OBJECT_ID, OBJECT_NUMBER, DECAYED についてはデータが入っていないことがある。

以下、dtype や convert_datesを指定せず型を自動判定させる場合の扱い。

JSON では欠損値は null となっている。
pandas.read_json の結果、nullは文字列の場合Noneに、数値の場合NaNに変換される。
このため整数しか入っていないカラムも、NaNを表現するためにfloat型となる場合がある。
OBJECT_ID については "ERROR" という文字列が入っている場合もある。

CSV では欠損値は単に長さ0の文字列となっている。
pandas.read_csv の結果、これらはいずれもNaNに変換される。
OBJECT_ID については "ERROR" という文字列が入っている場合もある。

XML では欠損値は \<DECAYED/\> のような空要素になっている。
変換方法によってどう扱われるかは異なる。
単純にDICT経由で変換する場合には長さ0の文字列になる。

In [22]:
# CSV
# OBJECT_NUMBER と DECAYED が float64 となっている (欠損値NaNを含むため)
df_csv = pd.read_csv('download/tle_latest_30day.csv')
df_csv.head(3)

,ORDINAL,COMMENT,ORIGINATOR,NORAD_CAT_ID,OBJECT_NAME,OBJECT_TYPE,CLASSIFICATION_TYPE,INTLDES,EPOCH,EPOCH_MICROSECONDS,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,FILE,TLE_LINE0,TLE_LINE1,TLE_LINE2,OBJECT_ID,OBJECT_NUMBER,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE,DECAYED
0,1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,5,VANGUARD 1,PAYLOAD,U,58002B,2020-11-07 14:35:09,256128,10.848685,0.184721,34.2558,147.2611,276.5774,62.8942,0,999,22080,-0.000043,-1.700000e-07,0.0,2875047,0 VANGUARD 1,1 5U 58002B 20312.60774602 -.00000017 +00000-0 -42903-4 0 9992,2 5 034.2558 147.2611 1847212 276.5774 062.8942 10.84868483220802,1958-002B,5.0,8619.533,132.734,3833.609,649.188,0.0
1,1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,11,VANGUARD 2,PAYLOAD,U,59001A,2020-11-07 18:30:28,328544,11.856864,0.146639,32.8703,237.3183,340.5303,14.4488,0,999,29199,0.000115,2.260000e-06,0.0,2875047,0 VANGUARD 2,1 11U 59001A 20312.77116121 +.00000226 +00000-0 +11540-3 0 9995,2 11 032.8703 237.3183 1466385 340.5303 014.4488 11.85686439291995,1959-001A,11.0,8123.726,121.448,2936.842,554.340,0.0
2,1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,12,VANGUARD R/B,ROCKET BODY,U,59001B,2020-11-07 19:07:28,619328,11.443499,0.166637,32.8968,209.4545,109.5610,269.1976,0,999,29640,0.000182,3.030000e-06,0.0,2875047,0 VANGUARD R/B,1 12U 59001B 20312.79685902 +.00000303 +00000-0 +18183-3 0 9992,2 12 032.8968 209.4545 1666368 109.5610 269.1976 11.44349894296406,1959-001B,12.0,8318.198,125.835,3326.181,553.945,0.0


In [23]:
# XML
# tle APIで取得するXMLはネストしていないので、単純にDICTを経由してDataFrameに変換する
import xml.etree.ElementTree as ET
import xmljson
data_dict = xmljson.yahoo.data(ET.fromstring(data_xml))
df_xml = pd.DataFrame(data_dict['xml']['item']) # 全てのデータが文字列型になる
#pd.read_json(json.dumps(data_dict['xml']['item'])) # 一旦文字列化してから read_json で変換することも可
df_xml.head(3)

,ORDINAL,COMMENT,ORIGINATOR,NORAD_CAT_ID,OBJECT_NAME,OBJECT_TYPE,CLASSIFICATION_TYPE,INTLDES,EPOCH,EPOCH_MICROSECONDS,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,FILE,TLE_LINE0,TLE_LINE1,TLE_LINE2,OBJECT_ID,OBJECT_NUMBER,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE,DECAYED
0,1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,5,VANGUARD 1,PAYLOAD,U,58002B,2020-11-07 14:35:09,256128,10.84868483,0.1847212,34.2558,147.2611,276.5774,62.8942,0,999,22080,-0.000042903,-0.00000017,0,2875047,0 VANGUARD 1,1 5U 58002B 20312.60774602 -.00000017 +00000-0 -42903-4 0 9992,2 5 034.2558 147.2611 1847212 276.5774 062.8942 10.84868483220802,1958-002B,5,8619.533,132.734,3833.609,649.188,0
1,1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,11,VANGUARD 2,PAYLOAD,U,59001A,2020-11-07 18:30:28,328544,11.85686439,0.1466385,32.8703,237.3183,340.5303,14.4488,0,999,29199,0.0001154,0.00000226,0,2875047,0 VANGUARD 2,1 11U 59001A 20312.77116121 +.00000226 +00000-0 +11540-3 0 9995,2 11 032.8703 237.3183 1466385 340.5303 014.4488 11.85686439291995,1959-001A,11,8123.726,121.448,2936.842,554.340,0
2,1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,12,VANGUARD R/B,ROCKET BODY,U,59001B,2020-11-07 19:07:28,619328,11.44349894,0.1666368,32.8968,209.4545,109.561,269.1976,0,999,29640,0.00018183,0.00000303,0,2875047,0 VANGUARD R/B,1 12U 59001B 20312.79685902 +.00000303 +00000-0 +18183-3 0 9992,2 12 032.8968 209.4545 1666368 109.5610 269.1976 11.44349894296406,1959-001B,12,8318.198,125.835,3326.181,553.945,0


In [24]:
# JSON
# OBJECT_NUMBER と DECAYED が float64 となっている (欠損値NaNを含むため)
df_json = pd.read_json('download/tle_latest_30day.json')
# df_json = pd.read_json(data_json) のように、JSON文字列を直接渡すことも可
df_json.head(3)

,ORDINAL,COMMENT,ORIGINATOR,NORAD_CAT_ID,OBJECT_NAME,OBJECT_TYPE,CLASSIFICATION_TYPE,INTLDES,EPOCH,EPOCH_MICROSECONDS,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,FILE,TLE_LINE0,TLE_LINE1,TLE_LINE2,OBJECT_ID,OBJECT_NUMBER,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE,DECAYED
0,1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,5,VANGUARD 1,PAYLOAD,U,58002B,2020-11-07 14:35:09,256128,10.848685,0.184721,34.2558,147.2611,276.5774,62.8942,0,999,22080,-0.000043,-1.700000e-07,0.0,2875047,0 VANGUARD 1,1 5U 58002B 20312.60774602 -.00000017 +00000-0 -42903-4 0 9992,2 5 034.2558 147.2611 1847212 276.5774 062.8942 10.84868483220802,1958-002B,5.0,8619.533,132.734,3833.609,649.188,0.0
1,1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,11,VANGUARD 2,PAYLOAD,U,59001A,2020-11-07 18:30:28,328544,11.856864,0.146639,32.8703,237.3183,340.5303,14.4488,0,999,29199,0.000115,2.260000e-06,0.0,2875047,0 VANGUARD 2,1 11U 59001A 20312.77116121 +.00000226 +00000-0 +11540-3 0 9995,2 11 032.8703 237.3183 1466385 340.5303 014.4488 11.85686439291995,1959-001A,11.0,8123.726,121.448,2936.842,554.340,0.0
2,1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,12,VANGUARD R/B,ROCKET BODY,U,59001B,2020-11-07 19:07:28,619328,11.443499,0.166637,32.8968,209.4545,109.5610,269.1976,0,999,29640,0.000182,3.030000e-06,0.0,2875047,0 VANGUARD R/B,1 12U 59001B 20312.79685902 +.00000303 +00000-0 +18183-3 0 9992,2 12 032.8968 209.4545 1666368 109.5610 269.1976 11.44349894296406,1959-001B,12.0,8318.198,125.835,3326.181,553.945,0.0


## 特定の衛星の過去の全データを取得する

In [25]:
# 特定の衛星の過去の全データ
data = st.tle(norad_cat_id=25544, orderby='epoch', format='json')

In [26]:
with open('download/tle_25544_all.json', 'w') as fp:
    fp.write(data)

In [27]:
df = pd.read_json(data)

In [28]:
# 従来のTLEの文字列も含まれている
print(df.TLE_LINE0[0])
print(df.TLE_LINE1[0])
print(df.TLE_LINE2[0])

0 ISS (ZARYA)
1 25544U 98067A   98324.28472222 -.00003657  11563-4  00000+0 0  9996
2 25544 051.5908 168.3788 0125362 086.4185 359.7454 16.05064833    05


In [29]:
# 複数の衛星の過去の全データをjsonで取得 (まとめて取得するので、同時にあまり多くの衛星を指定することはできない)
data = st.tle(norad_cat_id=[op.inclusive_range(1, 10)], orderby=['norad_cat_id', 'epoch'], format='json')

In [30]:
with open('download/tle_1-10_all.json', 'w') as fp:
        fp.write(data)

In [31]:
df = pd.read_json(data)

In [32]:
df

,COMMENT,ORIGINATOR,NORAD_CAT_ID,OBJECT_NAME,OBJECT_TYPE,CLASSIFICATION_TYPE,INTLDES,EPOCH,EPOCH_MICROSECONDS,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,FILE,TLE_LINE0,TLE_LINE1,TLE_LINE2,OBJECT_ID,OBJECT_NUMBER,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE,DECAYED
0,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-07-09 22:15:15,874847,13.063418,0.115556,33.1849,192.1350,327.9492,35.6894,0,999,3800,0.0,0.000806,0.0,33905,0 EXPLORER 1,1 00004U 58001 A 59190.92726707 .00080584 +00000-0 +00000-0 0 9997,2 00004 033.1849 192.1350 1155556 327.9492 035.6894 13.06341792038001,1958-001A,4,7615.485,110.231,2117.362,357.338,1
1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-07-17 13:46:31,735488,13.075958,0.114989,33.1849,157.3663,20.6576,341.5910,0,999,3900,0.0,0.000830,0.0,33905,0 EXPLORER 1,1 00004U 58001 A 59198.57397842 .00082991 +00000-0 +00000-0 0 9994,2 00004 033.1849 157.3663 1149895 020.6576 341.5910 13.07595833039007,1958-001A,4,7610.615,110.125,2107.621,357.339,1
2,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-08-01 20:45:25,598304,13.072853,0.115127,33.1849,87.4068,126.2272,239.3949,0,999,4100,0.0,0.000632,0.0,33905,0 EXPLORER 1,1 00004U 58001 A 59213.86487961 .00063193 +00000-0 +00000-0 0 9999,2 00004 033.1849 087.4068 1151275 126.2272 239.3949 13.07285348041006,1958-001A,4,7611.820,110.151,2110.015,357.355,1
3,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-08-17 03:10:46,984799,13.086318,0.114520,33.2199,16.0548,231.8066,133.1030,0,999,4300,0.0,0.000000,0.0,33905,0 EXPLORER 1,1 00004U 58001 A 59229.13248825 .00000000 +00000-0 +00000-0 0 9998,2 00004 033.2199 016.0548 1145204 231.8066 133.1030 13.08631791043007,1958-001A,4,7606.598,110.038,2099.573,357.352,1
4,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-09-09 00:23:47,124095,13.097691,0.113992,33.2139,269.8797,31.4052,331.9032,0,999,4600,0.0,0.000370,0.0,33905,0 EXPLORER 1,1 00004U 58001 A 59252.01651764 .00036992 +00000-0 +00000-0 0 9990,2 00004 033.2139 269.8797 1139920 031.4052 331.9032 13.09769139046004,1958-001A,4,7602.194,109.943,2090.648,357.469,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14410,GENERATED VIA SPACETRACK.ORG API,18 SPCS,9,EXPLORER 4,PAYLOAD,U,58005A,1959-08-30 13:01:54,971903,15.048483,0.044050,50.2549,243.3844,283.1585,79.2689,0,999,5570,0.0,0.004215,0.0,33905,0 EXPLORER 4,1 00009U 58005 A 59242.54299736 .00421521 +00000-0 +00000-0 0 9999,2 00009 050.2549 243.3844 0440497 283.1585 079.2689 15.04848316055702,1958-005A,9,6930.112,95.690,857.247,246.708,1
14411,GENERATED VIA SPACETRACK.ORG API,18 SPCS,9,EXPLORER 4,PAYLOAD,U,58005A,1959-09-06 04:00:31,256351,15.124788,0.040822,50.2549,211.3271,308.3519,53.4735,0,999,5670,0.0,0.005757,0.0,33905,0 EXPLORER 4,1 00009U 58005 A 59249.16702843 .00575697 +00000-0 +00000-0 0 9996,2 00009 050.2549 211.3271 0408223 308.3519 053.4735 15.12478751056707,1958-005A,9,6906.784,95.207,810.600,246.699,1
14412,GENERATED VIA SPACETRACK.ORG API,18 SPCS,9,EXPLORER 4,PAYLOAD,U,58005A,1959-09-11 10:35:46,68287,15.186702,0.038207,50.2549,185.5821,328.4908,32.6519,0,999,5750,0.0,0.006615,0.0,33905,0 EXPLORER 4,1 00009U 58005 A 59254.44150542 .00661498 +00000-0 +00000-0 0 9991,2 00009 050.2549 185.5821 0382066 328.4908 032.6519 15.18670171057505,1958-005A,9,6888.000,94.819,773.032,246.698,1
14413,GENERATED VIA SPACETRACK.ORG API,18 SPCS,9,EXPLORER 4,PAYLOAD,U,58005A,1959-09-27 18:24:17,445311,15.434091,0.028892,50.2549,104.6495,39.2935,321.7613,0,999,6000,0.0,0.008463,0.0,33905,0 EXPLORER 4,1 00009U 58005 A 59270.76686858 .00846266 +00000-0 +00000-0 0 9996,2 00009 050.2549 104.6495 0288920 039.2935 321.7613 15.43409094060003,1958-005A,9,6814.198,93.299,632.938,239.187,1
